# Exam

Develop a model for predicting review rating.  
**Multiclass classification into 5 classes**  
Score: **F1 with macro averaging**  
You are forbidden to use test dataset for any kind of training.  
Remember proper training pipeline.  
If you are not using default params in the models, you have to use some validation scheme to justify them. 

Use `random_state` or `seed` params - your experiment must be reprodusible.


### 1 baseline = 0.51
### 2 baseline = 0.53


In [0]:
SEED = 42

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!wget https://github.com/thedenaas/hse_seminars/blob/master/2019/exam/exam_data.zip
!unzip '/content/exam_data.zip'

In [0]:
import pandas as pd

In [44]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_train.shape

(48192, 3)

In [45]:
df_train.head()

,review,title,target
0,"The staff was very friendly, the breakfast ver...",Walker Gem,5
1,Excellent service - very approachable and prof...,Excellent Service,4
2,Really a top notch place to spend a day at the...,"Good location, warm and friendly staff",5
3,"a little noisy, there was a false fire alarm a...","nice hotel,",4
4,Place had too many animals and I'm allergic to...,Experience,3


In [6]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
import nltk
from nltk.corpus import stopwords
#set(stopwords.words('english'))

In [0]:
stopwords = set(stopwords.words('english'))

In [0]:
from nltk.tokenize import word_tokenize

In [0]:
def tokenizer(sent, no_stopwords=True):
  sent = word_tokenize(sent)
  if no_stopwords:
    sent_list = []
    for w in sent:
      if w not in stopwords:
        sent_list.append(w)
  else:
    sent_list = sent
  return sent_list

In [0]:
df_train['review_list'] = df_train['review'].apply(lambda x: tokenizer(x))
df_train['title_list'] = df_train['title'].apply(lambda x: tokenizer(x))

In [0]:
df_test['review_list'] = df_test['review'].apply(lambda x: tokenizer(x))
df_test['title_list'] = df_test['title'].apply(lambda x: tokenizer(x))

In [0]:
def make_dict(*cols):
  global word_dict, index_dict
  word_dict, index_dict = {}, {}
  word_dict['UNK'] = 0
  index_dict['0'] = 'UNK'
  i = 1
  for col in cols:
    for sent in col:
      for word in sent:
        if word not in word_dict.keys():
          word_dict[word] = i
          index_dict[i] = word
          i += 1
  return word_dict, index_dict

make_dict(df_train['review_list'], df_train['title_list'])

In [0]:
def make_num_sentence(sent, word_dict):
  num_sent = []
  for word in sent:
    num_sent.append(word_dict[word])
  return num_sent

In [0]:
df_train['review_nums'] = df_train['review_list'].apply(lambda x: make_num_sentence(x, word_dict))
df_train['title_nums'] = df_train['title_list'].apply(lambda x: make_num_sentence(x, word_dict))



In [0]:
df_test['review_length'] = df_test['review_list'].apply(lambda x: len(x))
df_test['title_length'] = df_test['title_list'].apply(lambda x: len(x))

In [49]:
from gensim.models.keyedvectors import KeyedVectors

import gensim.downloader as api
word_vectors = api.load("glove-wiki-gigaword-100")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
import numpy as np

In [0]:
def getting_fasttext(filepath):
    fasttext_model = KeyedVectors.load(filepath)
    return fasttext_model

def sent_vectorizer(sent, model): 

    lemmas_vectors = np.zeros((len(sent), model.vector_size)) 
    for idx, lemma in enumerate(sent): 
        if lemma in model.vocab: 
            lemmas_vectors[idx] = model.wv[lemma] 
    sent_vector = lemmas_vectors.mean(axis=0) 
    return sent_vector 

def fasttext_indexing(d):
    model = word_vectors
    vectors_dict = {}
    
    for idx, row in d.iterrows():
        sent_vec = sent_vectorizer(row.review_list, model)
        vectors_dict[idx] = sent_vec
        
    data = pd.DataFrame.from_dict(vectors_dict, orient='index')
    return data

In [52]:
df_train_vectors_r = fasttext_indexing(df_train[['review_list']])
df_test_vectors_r = fasttext_indexing(df_test[['review_list']])


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  # Remove the CWD from sys.path while we load stuff.


In [0]:
def fasttext_indexing(d):
    model = word_vectors
    vectors_dict = {}
    
    for idx, row in d.iterrows():
        sent_vec = sent_vectorizer(row.title_list, model)
        vectors_dict[idx] = sent_vec
        
    data = pd.DataFrame.from_dict(vectors_dict, orient='index')
    return data

In [53]:
df_train_vectors_t = fasttext_indexing(df_train[['title_list']])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  # Remove the CWD from sys.path while we load stuff.


In [56]:
df_test_vectors_t = fasttext_indexing(df_test[['title_list']])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: Mean of empty slice.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:154: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


In [0]:
from sklearn.decomposition import TruncatedSVD

In [0]:
svd = TruncatedSVD(n_components=10, n_iter=7, random_state=SEED)

In [33]:
#df_train_vectors_t.fillna(df_train_vectors_t.mean())

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,-0.431667,0.062893,0.274097,0.018545,-0.085472,0.017389,0.146246,0.315056,-0.286067,0.043705,-0.105503,-0.617797,0.406770,-0.002753,0.253302,-0.212054,0.181355,-0.333107,-0.279485,0.085587,0.014313,0.227944,0.195895,0.048733,-0.071945,-0.038975,-0.091613,-0.021744,0.064030,-0.076226,-0.179831,0.417229,-0.001327,-0.000873,0.148841,0.318177,0.017370,0.070743,-0.016703,-0.282467,...,-0.270545,0.482031,-0.046990,-0.059074,0.498145,-0.218714,0.235912,0.097888,0.339521,-0.047848,-0.035348,-0.215758,0.202270,-0.470755,0.350962,0.311658,0.226575,-0.133007,-0.216087,0.163623,0.365183,0.002406,-0.203425,0.088917,-0.747350,-0.025333,-0.006202,0.182291,-0.073721,-0.259243,0.256247,-0.101021,0.324049,0.099827,-0.462717,0.296426,-0.338313,-0.140888,0.187596,0.120798
3,0.045743,-0.186493,0.362227,-0.060547,0.081953,0.432997,0.286136,0.670643,0.026240,-0.112813,0.376897,-0.347500,-0.088063,0.248980,0.203200,-0.001057,0.011963,-0.234630,-0.395277,0.368837,0.378107,0.288925,0.071480,-0.358217,0.218087,-0.125910,-0.339573,-0.561010,-0.312207,-0.541517,-0.314903,0.236910,0.283767,0.083852,0.237843,0.586307,0.218223,0.384033,0.306113,-0.551270,...,-0.490067,0.480610,-0.415433,-0.135457,0.256690,-0.197095,0.481920,0.270821,0.078012,-0.107877,-0.275983,-0.351503,0.063148,-0.516620,0.174474,0.108357,0.400423,-0.084410,-0.472667,0.033673,0.463859,-0.246785,-0.090403,0.011221,-0.715847,-0.227771,0.008673,-0.106435,-0.283532,-0.486703,0.346383,-0.545100,0.511060,-0.049387,-0.832277,0.658623,-0.281739,-0.217781,0.542377,0.130140
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [0]:
#df_train_vectors_t.replace([np.inf, -np.inf], np.nan, inplace=True)

In [0]:
svd_reviews = svd.fit_transform(df_train_vectors_r)


In [0]:
#svd_titles = svd.fit_transform(df_train_vectors_t)

In [0]:
svd_reviews = pd.DataFrame.from_records(svd_reviews)
svd_titles = pd.DataFrame.from_records(df_train_vectors_t)#svd_titles)

In [0]:
svd_reviews_test = svd.fit_transform(df_test_vectors_r)


In [0]:
svd_reviews_test = pd.DataFrame.from_records(svd_reviews)
svd_titles_test = pd.DataFrame.from_records(df_test_vectors_t)#svd_titles)

In [0]:
def renaming(n, pref):
  rename_dict = {}
  for i in range(n):
    rename_dict[i] = f'{pref}_{i}'
  return rename_dict

rename_reviews = renaming(10, 'reviews')
rename_titles = renaming(100, 'titles')


In [65]:
train_df = pd.concat([df_train, svd_reviews], axis=1)
train_df.rename(columns=rename_reviews)


,review,title,target,review_list,title_list,reviews_0,reviews_1,reviews_2,reviews_3,reviews_4,reviews_5,reviews_6,reviews_7,reviews_8,reviews_9
0,"The staff was very friendly, the breakfast ver...",Walker Gem,5,"[The, staff, friendly, ,, breakfast, nice, ., ...","[Walker, Gem]",2.974426,-0.069387,0.378393,0.150578,0.375038,-0.289880,0.186859,-0.230023,-0.503101,-0.165334
1,Excellent service - very approachable and prof...,Excellent Service,4,"[Excellent, service, -, approachable, professi...","[Excellent, Service]",3.030604,-0.436086,-0.116102,-0.076074,0.125817,-0.211366,0.228639,-0.213325,0.115725,-0.097045
2,Really a top notch place to spend a day at the...,"Good location, warm and friendly staff",5,"[Really, top, notch, place, spend, day, beginn...","[Good, location, ,, warm, friendly, staff]",3.258713,-0.123356,-0.297114,-0.118052,0.132150,-0.023509,-0.028712,-0.030666,-0.084753,0.119213
3,"a little noisy, there was a false fire alarm a...","nice hotel,",4,"[little, noisy, ,, false, fire, alarm, midnigh...","[nice, hotel, ,]",3.377648,0.114495,-0.185578,-0.034461,-0.388714,-0.015352,0.144801,0.251999,0.082745,-0.024170
4,Place had too many animals and I'm allergic to...,Experience,3,"[Place, many, animals, I, 'm, allergic, pets, ...",[Experience],2.845775,0.067726,0.029010,-0.348187,-0.095190,-0.375173,0.029144,-0.048704,0.167147,-0.002943
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48187,"A friend of mine always books the cheapest, ba...","Comfy, cozy but oh, so grand!",4,"[A, friend, mine, always, books, cheapest, ,, ...","[Comfy, ,, cozy, oh, ,, grand, !]",2.521485,0.105997,-0.234093,-0.088658,0.003830,-0.059303,-0.078726,0.139586,0.035069,-0.047618
48188,Stayed here with my family over Spring Break i...,Great location and price for family lodging.,4,"[Stayed, family, Spring, Break, 2-bedroom, sui...","[Great, location, price, family, lodging, .]",2.541392,-0.018298,0.080100,0.237153,0.088036,-0.164105,-0.005437,0.073956,0.053013,0.067062
48189,One word AWFUL and the pool was closed,Good hotel in a quiet part of Memphis,2,"[One, word, AWFUL, pool, closed]","[Good, hotel, quiet, part, Memphis]",1.896316,-0.104776,0.266972,0.266516,-0.229428,0.195658,-0.020981,-0.050969,0.237595,0.138702
48190,Never will stay here again. Dirty towels shirt...,Filthy,1,"[Never, stay, ., Dirty, towels, shirts, behind...",[Filthy],2.169894,0.170527,0.637581,0.088574,-0.624435,-0.156549,0.062989,0.089401,0.057276,-0.074481


In [0]:
train_df = pd.concat([train_df, svd_titles], axis=1)
train_df.rename(columns=rename_titles)
train_df = train_df.dropna(axis=0)

In [0]:
X_train = train_df.drop(columns=['review', 'title', 'target', 'review_list', 'title_list'])
y_train = train_df['target']

In [136]:
X_train.shape

(48186, 210)

In [129]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=SEED)

model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

# Testing

In [0]:
test_df = pd.concat([df_test, svd_reviews_test], axis=1)
test_df.rename(columns=rename_reviews)
test_df = pd.concat([test_df, svd_titles_test], axis=1)
test_df.rename(columns=rename_titles)
test_df = test_df.dropna(axis=0)


In [0]:
X_test = test_df.drop(columns=['review', 'title', 'target', 'review_list', 'title_list'])
X_test = X_test.dropna(axis=0)
y_test = test_df['target']

In [0]:
X_test = X_test.drop(columns=['review_length', 'title_length'])
#X_test = X_test.drop(columns=['review', 'title', 'target', 'review_list', 'title_list'])


In [0]:
X_test = X_test.replace([np.inf, -np.inf], np.nan, inplace=True)


In [135]:
X_test.shape, y_test.shape

((5354, 110), (5354,))

In [0]:
from sklearn import metrics

In [134]:
#print('train', metrics.f1_score(y_train, np.around(model.predict(X_train), 0), average='macro'))
print('test', metrics.f1_score(y_test, np.around(model.predict(X_test), 0), average='macro'))

ValueError: ignored